# File I/O & Serialization

**Level 2: Intermediate - Advanced Data Persistence & Security**

**Master professional data persistence, binary serialization, and secure file operations**

---

## Advanced File Operations

**Professional file handling with NIO.2, random access, and buffered streams**

In [ ]:
// Professional advanced file operations
import java.io.*;
import java.nio.*;
import java.nio.channels.*;
import java.nio.file.*;
import java.nio.charset.*;
import java.util.*;
import java.util.stream.*;

public class AdvancedFileOperations {

    public static void demonstrateRandomAccessFiles() {
        System.out.println("=== RANDOM ACCESS FILE OPERATIONS ===\n");

        Path filePath = Paths.get("random_access_data.dat");

        try (RandomAccessFile raf = new RandomAccessFile(filePath.toFile(), "rw")) {

            // Write data at different positions
            System.out.println("Writing data to random positions...");

            // Position 0: Customer ID
            raf.writeInt(1001); // Customer ID
            raf.writeUTF("John Doe"); // Name

            // Position 50: Account balance
            raf.seek(50);
            raf.writeDouble(5000.50);

            // Position 100: Transaction count
            raf.seek(100);
            raf.writeInt(25); // Transaction count

            // Position 150: Last access date
            raf.seek(150);
            raf.writeLong(System.currentTimeMillis());

            System.out.println("✅ Data written to random file positions");

            // Read data from specific positions
            System.out.println("\nReading data from specific positions:");

            raf.seek(0);
            int customerId = raf.readInt();
            String name = raf.readUTF();

            raf.seek(50);
            double balance = raf.readDouble();

            raf.seek(100);
            int transactionCount = raf.readInt();

            raf.seek(150);
            long lastAccess = raf.readLong();

            System.out.printf("Customer #%d: %s, Balance: $%.2f, Transactions: %d, Last Access: %tc\n",
                            customerId, name, balance, transactionCount, new Date(lastAccess));

            // Update balance in place
            raf.seek(50);
            raf.writeDouble(balance + 1000.00);

            raf.seek(50);
            double newBalance = raf.readDouble();
            System.out.printf("✅ Updated balance: $%.2f\n", newBalance);

        } catch (IOException e) {
            System.out.println("❌ File operation error: " + e.getMessage());
        }

        // Clean up
        try {
            Files.deleteIfExists(filePath);
            System.out.println("\nCleaned up temporary file\n");
        } catch (IOException e) {
            System.out.println("Could not delete file: " + e.getMessage());
        }
    }

    public static void demonstrateMappedByteBuffers() {
        System.out.println("=== MEMORY-MAPPED FILE I/O (HIGH PERFORMANCE) ===\n");

        Path filePath = Paths.get("memory_mapped.txt");

        try {
            // Create a test file with some content
            List<String> lines = Arrays.asList(
                "Memory-mapped I/O provides direct access to file content",
                "through virtual memory mapping for maximum performance.",
                "This is faster than traditional stream-based I/O for large files."
            );

            Files.write(filePath, lines, StandardOpenOption.CREATE);

            // Memory-map the file for reading
            try (FileChannel fileChannel = FileChannel.open(filePath, StandardOpenOption.READ)) {
                MappedByteBuffer buffer = fileChannel.map(FileChannel.MapMode.READ_ONLY, 0, fileChannel.size());

                System.out.println("File content via memory-mapped buffer:");
                System.out.println("-----------------------------------------");

                // Decode bytes to string
                String content = StandardCharsets.UTF_8.decode(buffer).toString();
                System.out.println(content);

                // Demonstrate random access
                System.out.println("\nRandom access demonstration:");
                buffer.position(0);
                byte firstByte = buffer.get();
                System.out.println("First byte: " + (char) firstByte);

                buffer.position(10);
                byte eleventhByte = buffer.get();
                System.out.println("11th byte: " + (char) eleventhByte);
            }

            System.out.println("\n✅ Memory-mapped I/O completed (best for large files)");

            // Cleanup
            Files.deleteIfExists(filePath);

        } catch (IOException e) {
            System.out.println("❌ Memory mapped I/O error: " + e.getMessage());
        }
    }

    public static void demonstrateAdvancedFileOperations() {
        System.out.println("=== ADVANCED FILE OPERATIONS WITH NIO.2 ===\n");

        Path basePath = Paths.get("advanced_file_demo");

        try {
            // Create directory structure
            Files.createDirectories(basePath.resolve("data"));
            Files.createDirectories(basePath.resolve("logs"));
            Files.createDirectories(basePath.resolve("temp"));

            // Create test files with different attributes
            Path dataFile = basePath.resolve("data/customer_data.txt");
            Path logFile = basePath.resolve("logs/application.log");
            Path tempFile = basePath.resolve("temp/temp_data.tmp");

            // Write content with charset specification
            List<String> dataContent = Arrays.asList(
                "Customer Data - UTF-8 Encoded",
                "ID: 1001, Name: Alice, Balance: $2500.00",
                "Timestamp: " + new Date()
            );

            Files.write(dataFile, dataContent, StandardCharsets.UTF_8,
                       StandardOpenOption.CREATE, StandardOpenOption.TRUNCATE_EXISTING);

            // Write log entries (append mode)
            List<String> logEntries = Arrays.asList(
                "[" + new Date() + "] INFO: Application started",
                "[" + new Date() + "] INFO: Loaded customer data",
                "[" + new Date() + "] INFO: Processing completed successfully"
            );

            Files.write(logFile, logEntries, StandardOpenOption.CREATE, StandardOpenOption.APPEND);

            // Create temporary file that self-destructs
            Files.write(tempFile, "Temporary data".getBytes());
            Files.deleteIfExists(tempFile); // Immediate cleanup

            System.out.println("Directory structure created:");
            System.out.println(basePath.toAbsolutePath());

            // Demonstrate file system walking
            System.out.println("\nFile system contents:");
            try (Stream<Path> paths = Files.walk(basePath)) {
                paths.filter(Files::isRegularFile)
                     .forEach(path -> {
                         try {
                             long size = Files.size(path);
                             System.out.println("  " + basePath.relativize(path) + " (" + size + " bytes)");
                         } catch (IOException e) {
                             System.out.println("  " + basePath.relativize(path) + " (error getting size)");
                         }
                     });
            }

            // Demonstrate file attributes
            System.out.println("\nFile attributes:");
            for (Path file : Arrays.asList(dataFile, logFile)) {
                try {
                    FileTime lastModified = Files.getLastModifiedTime(file);
                    boolean isReadable = Files.isReadable(file);
                    boolean isWritable = Files.isWritable(file);

                    System.out.println("  " + file.getFileName() +
                                     " - Modified: " + lastModified +
                                     " - Readable: " + isReadable +
                                     " - Writable: " + isWritable);
                } catch (IOException e) {
                    System.out.println("  " + file.getFileName() + " - error reading attributes");
                }
            }

            System.out.println("\n✅ Advanced file operations completed successfully");

            // Batch cleanup
            try (Stream<Path> cleanup = Files.walk(basePath)) {
                cleanup.sorted(Comparator.reverseOrder())
                       .forEach(path -> {
                           try {
                               Files.deleteIfExists(path);
                           } catch (IOException e) {
                               System.out.println("Could not delete: " + path);
                           }
                       });
            }

        } catch (IOException e) {
            System.out.println("❌ Advanced file operations error: " + e.getMessage());
        }
    }

    public static void main(String[] args) {
        demonstrateRandomAccessFiles();
        demonstrateMappedByteBuffers();
        demonstrateAdvancedFileOperations();

        System.out.println("\n🎯 ADVANCED FILE OPERATIONS MASTERED:");
        System.out.println("• Random access file positioning and updates");
        System.out.println("• Memory-mapped I/O for high performance");
        System.out.println("• NIO.2 advanced file system operations");
        System.out.println("• Directory walking and attribute management");
        System.out.println("• Charset-aware file reading/writing");
        System.out.println("• Batch file operations and cleanup");
    }
}


## Object Serialization Fundamentals

**Binary object persistence with automatic serialization**

In [ ]:
// Object serialization - saving/restoring complete object graphs
import java.io.*;
import java.time.LocalDate;
import java.util.*;

public class ObjectSerialization {

    // Serializable business objects
    static class Customer implements Serializable {
        private static final long serialVersionUID = 1L;

        private int customerId;
        private String name;
        private String email;
        private LocalDate registrationDate;
        private List<Order> orders;

        public Customer(int customerId, String name, String email) {
            this.customerId = customerId;
            this.name = name;
            this.email = email;
            this.registrationDate = LocalDate.now();
            this.orders = new ArrayList<>();
        }

        public void addOrder(Order order) {
            this.orders.add(order);
            order.setCustomer(this); // Bidirectional relationship
        }

        @Override
        public String toString() {
            return String.format("Customer{id=%d, name='%s', orders=%d}",
                               customerId, name, orders.size());
        }
    }

    static class Order implements Serializable {
        private static final long serialVersionUID = 1L;

        private int orderId;
        private double amount;
        private List<OrderItem> items;
        private transient Customer customer; // Not serialized

        public Order(int orderId, double amount) {
            this.orderId = orderId;
            this.amount = amount;
            this.items = new ArrayList<>();
        }

        public void addItem(OrderItem item) {
            this.items.add(item);
        }

        public void setCustomer(Customer customer) {
            this.customer = customer;
        }

        @Override
        public String toString() {
            return String.format("Order{id=%d, amount=%.2f, items=%d}",
                               orderId, amount, items.size());
        }
    }

    static class OrderItem implements Serializable {
        private static final long serialVersionUID = 1L;

        private String productName;
        private int quantity;
        private double unitPrice;

        public OrderItem(String productName, int quantity, double unitPrice) {
            this.productName = productName;
            this.quantity = quantity;
            this.unitPrice = unitPrice;
        }

        @Override
        public String toString() {
            return String.format("OrderItem{product='%s', qty=%d, price=%.2f}",
                               productName, quantity, unitPrice);
        }
    }

    public static void demonstrateBasicSerialization() {
        System.out.println("=== BASIC OBJECT SERIALIZATION ===\n");

        // Create complex object graph
        Customer customer = new Customer(1001, "Alice Johnson", "alice@example.com");

        // Create orders with items
        Order order1 = new Order(2001, 150.00);
        order1.addItem(new OrderItem("Laptop", 1, 1200.00));
        order1.addItem(new OrderItem("Mouse", 2, 25.00));

        Order order2 = new Order(2002, 75.50);
        order2.addItem(new OrderItem("Keyboard", 1, 75.50));

        customer.addOrder(order1);
        customer.addOrder(order2);

        System.out.println("Original customer data:");
        System.out.println(customer);
        System.out.println("Orders:");
        customer.orders.forEach(order -> {
            System.out.println("  " + order);
            order.items.forEach(item -> System.out.println("    " + item));
        });

        // Serialize to file
        Path serializedFile = Paths.get("customer_data.ser");

        try (FileOutputStream fos = new FileOutputStream(serializedFile.toFile());
             ObjectOutputStream oos = new ObjectOutputStream(fos)) {

            oos.writeObject(customer);
            System.out.println("\n✅ Customer object serialized successfully");

        } catch (IOException e) {
            System.out.println("❌ Serialization failed: " + e.getMessage());
            return;
        }

        // Deserialize from file
        try (FileInputStream fis = new FileInputStream(serializedFile.toFile());
             ObjectInputStream ois = new ObjectInputStream(fis)) {

            Customer deserializedCustomer = (Customer) ois.readObject();

            System.out.println("\n✅ Customer object deserialized successfully");
            System.out.println("Deserialized customer data:");
            System.out.println(deserializedCustomer);
            System.out.println("Orders:");
            deserializedCustomer.orders.forEach(order -> {
                System.out.println("  " + order);
                order.items.forEach(item -> System.out.println("    " + item));
            });

            // Note: transient customer reference in Order is null
            System.out.println("\n⚠️  Note: Customer references in orders are null (transient fields)");

        } catch (IOException | ClassNotFoundException e) {
            System.out.println("❌ Deserialization failed: " + e.getMessage());
        }

        // Cleanup
        try {
            Files.deleteIfExists(serializedFile);
        } catch (IOException e) {
            System.out.println("Could not delete serialized file");
        }
    }

    public static void demonstrateTransientFields() {
        System.out.println("\n=== TRANSIENT FIELDS DEMONSTRATION ===\n");

        static class PasswordHolder implements Serializable {
            private static final long serialVersionUID = 1L;

            private String username;
            private transient String password; \n            private String securityQuestion;
            private transient String answer;     
            private boolean accountActive;

            public PasswordHolder(String username, String password) {
                this.username = username;
                this.password = password;
                this.securityQuestion = "Mother's maiden name?";
                this.answer = "Smith";
                this.accountActive = true;
            }

            public String toString() {
                return String.format("User{username='%s', password='%s', question='%s', answer='%s', active=%s}",
                                   username, password, securityQuestion, answer, accountActive);
            }
        }

        PasswordHolder user = new PasswordHolder("admin", "secret123");

        System.out.println("Original object:");
        System.out.println(user);
        
        // Serialize and deserialize
        Path filePath = Paths.get("user_transient.ser");

        try (ObjectOutputStream oos = new ObjectOutputStream(Files.newOutputStream(filePath));
             ObjectInputStream ois = new ObjectInputStream(Files.newInputStream(filePath))) {

            // Serialize
            oos.writeObject(user);
            
            // Deserialize
            PasswordHolder deserializedUser = (PasswordHolder) ois.readObject();
            
            System.out.println("\nAfter deserialization (transient fields are null):");
            System.out.println(deserializedUser);
            
            System.out.println("\n🎯 TRANSIENT FIELDS:");
            System.out.println("• Fields marked transient are not serialized");
            System.out.println("• Security: Don't serialize passwords, keys, sensitive data");
            System.out.println("• Performance: Skip large objects that can be reconstructed");
            System.out.println("• Flexibility: Skip derived/computed fields");

        } catch (IOException | ClassNotFoundException e) {
            System.out.println("❌ Transient demo failed: " + e.getMessage());
        }

        // Cleanup
        try {
            Files.deleteIfExists(filePath);
        } catch (IOException e) {
            System.out.println("Could not delete file");
        }
    }

    public static void main(String[] args) {
        demonstrateBasicSerialization();
        demonstrateTransientFields();

        System.out.println("\n🎯 OBJECT SERIALIZATION MASTERED:");
        System.out.println("• Complete object graphs persistence");
        System.out.println("• Automatic serialization with Serializable");
        System.out.println("• Transient fields for security/performance");
        System.out.println("• serialVersionUID for version compatibility");
        System.out.println("• Binary format for efficiency and speed");
    }
}
